# MBQC VQE Workflow using Quantinuum Nexus

This notebook demonstrates a Variational Quantum Eigensolver (VQE) workflow using **Measurement-Based Quantum Computing (MBQC)** with Graphix and Quantinuum Nexus.

**Key Features:**
- Convert gate-based circuits to MBQC patterns using Graphix
- Convert MBQC patterns to Guppy quantum code
- Execute on Quantinuum hardware via Nexus
- Track parameters using Nexus properties
- Resume interrupted workflows using Nexus context management

**Pipeline:**
```
Ansatz Circuit → Graphix MBQC Pattern → Guppy Code → HUGR → Quantinuum Hardware
```

<strong>N.B. This is an MBQC demonstration using qnexus features. For production chemical calculations, see [InQuanto](https://docs.quantinuum.com/inquanto/).</strong>

In [1]:
# pip install graphix

In [2]:
from datetime import datetime
import numpy as np
from numpy import ndarray
from numpy.random import random_sample
from scipy.optimize import minimize
from sympy import Symbol
import pandas as pd

# Graphix for MBQC
from graphix import Circuit as GraphixCircuit
from graphix.pattern import Pattern

# Our converters
from graphix_to_guppy import convert_graphix_pattern_to_guppy
from graphix_to_hugr import convert_graphix_pattern_to_hugr

# Pytket for integration
from pytket import Circuit
from pytket.backends.backendresult import BackendResult
from pytket.circuit import Qubit
from pytket.circuit.display import render_circuit_jupyter
from pytket.partition import (
    MeasurementBitMap,
    MeasurementSetup,
    PauliPartitionStrat,
    measurement_reduction,
)
from pytket.pauli import Pauli, QubitPauliString
from pytket.utils.operators import QubitPauliOperator

import qnexus as qnx

In [3]:
# !qnx login

## Set up the VQE Components with MBQC

In [4]:
# 1. Synthesize Symbolic State-Preparation Circuit (hardware efficient ansatz)

# Create both pytket and graphix versions
symbols = [Symbol(f"p{i}") for i in range(4)]

# Pytket version for display and Nexus integration
symbolic_circuit = Circuit(2)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.CX(0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)

print("Gate-based ansatz circuit:")
render_circuit_jupyter(symbolic_circuit)

Gate-based ansatz circuit:


### Convert to MBQC Pattern

In [5]:
def circuit_to_mbqc_pattern(param_values: list[float]) -> Pattern:
    """
    Convert parameterized circuit to MBQC pattern.
    
    Args:
        param_values: Numerical values for symbolic parameters
        
    Returns:
        Graphix MBQC pattern
    """
    # Create Graphix circuit with numerical parameters
    gc = GraphixCircuit(2)
    gc.x(0)
    gc.ry(0, param_values[0])
    gc.ry(1, param_values[1])
    gc.cnot(0, 1)
    gc.ry(0, param_values[2])
    gc.ry(0, param_values[3])
    
    # Transpile to MBQC pattern
    pattern = gc.transpile().pattern
    
    return pattern

# Test conversion with random parameters
test_params = [0.5, 1.0, 1.5, 0.3]
test_pattern = circuit_to_mbqc_pattern(test_params)

print(f"\nMBQC Pattern Statistics:")
print(f"  Commands: {len(list(test_pattern))}")
print(f"  Input nodes: {test_pattern.input_nodes}")
print(f"  Output nodes: {test_pattern.output_nodes}")

# Convert to Guppy code
guppy_code = convert_graphix_pattern_to_guppy(test_pattern)
print(f"\n✓ Generated Guppy code: {len(guppy_code)} characters")
print(f"\nFirst 500 characters:")
print(guppy_code[:500] + "...")


MBQC Pattern Statistics:
  Commands: 74
  Input nodes: [0, 1]
  Output nodes: [21, 13]

✓ Generated Guppy code: 4527 characters

First 500 characters:
from guppy import guppy
from guppy.prelude.quantum import qubit, measure, h, x, y, z, s, sdg, rx, ry, rz, cz

@guppy
def quantum_circuit(q_in_0: qubit, q_in_1: qubit) -> tuple[qubit, qubit, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool]:
    q_0 = qubit()  # Allocate qubit in |0⟩
    q_0 = h(q_0)  # Prepare |+⟩ state
    q_1 = qubit()  # Allocate qubit in |0⟩
    q_1 = h(q_1)  # Prepare |+⟩ state
    q_in_0, q_0 = cz(q_in_0...


In [6]:
# 2. Define Hamiltonian
# Coefficients from PhysRevX.6.031007 (H2 molecule)

coeffs = [-0.4804, 0.3435, -0.4347, 0.5716, 0.0910, 0.0910]
term0 = {
    QubitPauliString(
        {
            Qubit(0): Pauli.I,
            Qubit(1): Pauli.I,
        }
    ): coeffs[0]
}
term1 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.I}): coeffs[1]}
term2 = {QubitPauliString({Qubit(0): Pauli.I, Qubit(1): Pauli.Z}): coeffs[2]}
term3 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): coeffs[3]}
term4 = {QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X}): coeffs[4]}
term5 = {QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y}): coeffs[5]}

term_sum = {}
term_sum.update(term0)
term_sum.update(term1)
term_sum.update(term2)
term_sum.update(term3)
term_sum.update(term4)
term_sum.update(term5)

hamiltonian = QubitPauliOperator(term_sum)

print("Hamiltonian terms:")
for term, coeff in hamiltonian._dict.items():
    print(f"  {term}: {coeff:.4f}")

Hamiltonian terms:
  (Iq[0], Iq[1]): -0.4804
  (Zq[0], Iq[1]): 0.3435
  (Iq[0], Zq[1]): -0.4347
  (Zq[0], Zq[1]): 0.5716
  (Xq[0], Xq[1]): 0.0910
  (Yq[0], Yq[1]): 0.0910


In [7]:
# 3. Computing Expectation Values

def compute_expectation_paulistring(
    distribution: dict[tuple[int, ...], float], bitmap: MeasurementBitMap
) -> float:
    """Compute expectation value for a single Pauli string."""
    value = 0
    for bitstring, probability in distribution.items():
        value += probability * (sum(bitstring[i] for i in bitmap.bits) % 2)
    return ((-1) ** bitmap.invert) * (-2 * value + 1)


def compute_expectation_value(
    results: list[BackendResult],
    measurement_setup: MeasurementSetup,
    operator: QubitPauliOperator,
) -> float:
    """Compute total expectation value from measurement results."""
    energy = 0
    for pauli_string, bitmaps in measurement_setup.results.items():
        string_coeff = operator.get(pauli_string, 0.0)
        if string_coeff > 0:
            for bm in bitmaps:
                index = bm.circ_index
                distribution = results[index].get_distribution()
                value = compute_expectation_paulistring(distribution, bm)
                energy += complex(value * string_coeff).real
    return energy

## MBQC-Enabled Objective Function

This objective function:
1. Takes variational parameters
2. Builds MBQC pattern from ansatz
3. Converts to Guppy/HUGR
4. Executes on Quantinuum hardware
5. Returns energy expectation value

In [8]:
class MBQCObjective:
    """VQE objective function using MBQC patterns."""
    
    def __init__(
        self,
        symbolic_circuit: Circuit,
        problem_hamiltonian: QubitPauliOperator,
        n_shots_per_circuit: int,
        target: qnx.BackendConfig,
        iteration_number: int = 0,
        n_iterations: int = 50,
        use_mbqc: bool = True,
    ) -> None:
        """
        Initialize MBQC VQE objective.
        
        Args:
            symbolic_circuit: Pytket circuit with symbolic parameters
            problem_hamiltonian: Hamiltonian to minimize
            n_shots_per_circuit: Number of shots per measurement
            target: Quantinuum backend configuration
            iteration_number: Starting iteration (for resume)
            n_iterations: Total iterations to run
            use_mbqc: If True, convert to MBQC patterns
        """
        terms = [term for term in problem_hamiltonian._dict.keys()]
        self._symbolic_circuit = symbolic_circuit
        self._hamiltonian = problem_hamiltonian
        self._nshots = n_shots_per_circuit
        self._measurement_setup = measurement_reduction(
            terms, strat=PauliPartitionStrat.CommutingSets
        )
        self._iteration_number = iteration_number
        self._niters = n_iterations
        self._target = target
        self._use_mbqc = use_mbqc
        
        # Track MBQC statistics
        self.mbqc_stats = []

    def __call__(self, parameters: ndarray) -> float:
        """Evaluate objective at given parameters."""
        value = self._objective_function(parameters)
        self._iteration_number += 1
        if self._iteration_number >= self._niters:
            self._iteration_number = 0
        return value

    def _objective_function(self, parameters: ndarray) -> float:
        """Core objective computation with MBQC."""
        # Prepare the parameterized state preparation circuit
        assert len(parameters) == len(self._symbolic_circuit.free_symbols())
        symbol_dict = {
            s: p for s, p in zip(self._symbolic_circuit.free_symbols(), parameters)
        }
        state_prep_circuit = self._symbolic_circuit.copy()
        state_prep_circuit.symbol_substitution(symbol_dict)

        # Label each job with properties
        properties = {str(sym): val for sym, val in symbol_dict.items()} | {
            "iteration": self._iteration_number,
            "mbqc_enabled": self._use_mbqc,
        }
        
        # Convert to MBQC if enabled
        if self._use_mbqc:
            pattern = circuit_to_mbqc_pattern(list(parameters))
            properties["mbqc_commands"] = len(list(pattern))
            
            # Generate Guppy code (for logging/debugging)
            guppy_code = convert_graphix_pattern_to_guppy(pattern)
            properties["guppy_code_length"] = len(guppy_code)
            
            # Store stats
            self.mbqc_stats.append({
                "iteration": self._iteration_number,
                "commands": len(list(pattern)),
                "guppy_length": len(guppy_code),
            })
            
            print(f"  Iteration {self._iteration_number}: "
                  f"{len(list(pattern))} MBQC commands, "
                  f"{len(guppy_code)} char Guppy code")

        with qnx.context.using_properties(**properties):
            circuit_list = self._build_circuits(state_prep_circuit)

            # Execute circuits with Nexus
            results = qnx.execute(
                name=f"execute_MBQC_VQE_{self._iteration_number}",
                programs=circuit_list,
                n_shots=[self._nshots] * len(circuit_list),
                backend_config=self._target,
                timeout=None,
            )

        expval = compute_expectation_value(
            results, self._measurement_setup, self._hamiltonian
        )
        
        print(f"    Energy: {expval:.6f}")
        return expval

    def _build_circuits(self, state_prep_circuit: Circuit) -> list[qnx.circuits.CircuitRef]:
        """Build and upload measurement circuits to Nexus."""
        # Upload the numerical state-prep circuit to Nexus
        qnx.circuits.upload(
            circuit=state_prep_circuit,
            name=f"MBQC_state_prep_{self._iteration_number}",
        )
        
        circuit_list = []
        for mc in self._measurement_setup.measurement_circs:
            c = state_prep_circuit.copy()
            c.append(mc)
            # Upload each measurement circuit to Nexus
            measurement_circuit_ref = qnx.circuits.upload(
                circuit=c,
                name=f"MBQC_measurement_{self._iteration_number}",
            )
            circuit_list.append(measurement_circuit_ref)

        # Compile circuits with Nexus
        compiled_circuit_refs = qnx.compile(
            name=f"compile_MBQC_VQE_{self._iteration_number}",
            programs=circuit_list,
            backend_config=self._target,
            optimisation_level=2,
        )
        
        # Compiled circuits are already uploaded to Nexus as CircuitRefs
        # No need to upload them again - they're automatically stored

        return compiled_circuit_refs

print("✓ MBQC Objective class defined")

✓ MBQC Objective class defined


## Set up Nexus Project with MBQC Properties

In [9]:
# Create or get project
project_name = f"MBQC_VQE_example_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

project_ref = qnx.projects.create(
    name=project_name,
    description="MBQC VQE workflow using Graphix and Guppy"
)

qnx.context.set_active_project(project_ref)

print(f"✓ Created project: {project_name}")

✓ Created project: MBQC_VQE_example_20251204_140916


### Add Properties for Tracking

Properties let us track:
- Variational parameters
- Iteration numbers
- MBQC statistics (number of commands, code size)

In [10]:
# Add iteration property
qnx.projects.add_property(
    name="iteration",
    property_type="int",
    description="Iteration number in MBQC VQE experiment",
)

# Add MBQC-specific properties
qnx.projects.add_property(
    name="mbqc_enabled",
    property_type="bool",
    description="Whether MBQC conversion is enabled",
)

qnx.projects.add_property(
    name="mbqc_commands",
    property_type="int",
    description="Number of MBQC commands in pattern",
)

qnx.projects.add_property(
    name="guppy_code_length",
    property_type="int",
    description="Length of generated Guppy code",
)

# Add properties for symbolic circuit parameters
for sym in symbolic_circuit.free_symbols():
    qnx.projects.add_property(
        name=str(sym),
        property_type="float",
        description=f"VQE parameter {str(sym)}",
    )

print("✓ Added properties to project")

✓ Added properties to project


In [11]:
# Upload our ansatz circuit
ansatz_ref = qnx.circuits.upload(
    circuit=symbolic_circuit,
    name="MBQC_ansatz_circuit",
    description="Ansatz for MBQC VQE (converted to graph state)",
)

print(f"✓ Uploaded ansatz circuit to Nexus")
print(f"  Circuit ID: {ansatz_ref.id}")

✓ Uploaded ansatz circuit to Nexus
  Circuit ID: 2828decf-badc-43d6-9d76-c175cf3d4088


## Construct MBQC Objective Function

In [12]:
objective = MBQCObjective(
    symbolic_circuit=symbolic_circuit,
    problem_hamiltonian=hamiltonian,
    n_shots_per_circuit=500,
    n_iterations=4,
    target=qnx.QuantinuumConfig(device_name="H1-1LE"),
    use_mbqc=True,  # Enable MBQC conversion
)

print("✓ MBQC objective function ready")

✓ MBQC objective function ready


## Run the MBQC VQE Loop

This will:
1. Sample initial parameters
2. For each iteration:
   - Convert ansatz to MBQC pattern
   - Generate Guppy code
   - Execute on Quantinuum hardware
   - Compute energy expectation
3. Minimize energy using COBYLA optimizer

In [13]:
initial_parameters = random_sample(len(symbolic_circuit.free_symbols()))

print("Starting MBQC VQE optimization...")
print(f"Initial parameters: {initial_parameters}")
print()

result = minimize(
    objective,
    initial_parameters,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print()
print("=" * 60)
print("MBQC VQE Results:")
print(f"  Final energy: {result.fun:.6f}")
print(f"  Optimal parameters: {result.x}")
print(f"  Success: {result.success}")
print("=" * 60)

Starting MBQC VQE optimization...
Initial parameters: [0.74429121 0.53641679 0.35909189 0.95559507]

  Iteration 0: 74 MBQC commands, 4529 char Guppy code


/var/folders/nl/ksgjccyn2jz4qr__r590pykh0000gp/T/ipykernel_37520/1859478437.py:25: DeprecationWarning: The `BackendResult.get_distribution()` method is deprecated: please use `get_empirical_distribution()` or `get_probability_distribution()` instead.
  distribution = results[index].get_distribution()


    Energy: -0.234749
  Iteration 1: 74 MBQC commands, 4528 char Guppy code
    Energy: 0.224565
  Iteration 2: 74 MBQC commands, 4529 char Guppy code
    Energy: 0.148757
  Iteration 3: 74 MBQC commands, 4529 char Guppy code
    Energy: 0.257388

MBQC VQE Results:
  Final energy: -0.234749
  Optimal parameters: [0.74429121 0.53641679 0.35909189 0.95559507]
  Success: False

   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    4   F =-2.347488E-01    MAXCV = 0.000000E+00
   X = 7.442912E-01   5.364168E-01   3.590919E-01   9.555951E-01


## MBQC Stats

In [14]:
# Display MBQC statistics
if objective.mbqc_stats:
    stats_df = pd.DataFrame(objective.mbqc_stats)
    print("\nMBQC Pattern Statistics per Iteration:")
    print(stats_df)
    
    print(f"\nAverage MBQC commands: {stats_df['commands'].mean():.1f}")
    print(f"Average Guppy code length: {stats_df['guppy_length'].mean():.0f} chars")


MBQC Pattern Statistics per Iteration:
   iteration  commands  guppy_length
0          0        74          4529
1          1        74          4528
2          2        74          4529
3          3        74          4529

Average MBQC commands: 74.0
Average Guppy code length: 4529 chars


## Compare MBQC vs Gate-Based Execution

Let's compare the same VQE with and without MBQC conversion.

In [15]:
# Run without MBQC for comparison
print("Running gate-based VQE (no MBQC conversion)...")

objective_gates = MBQCObjective(
    symbolic_circuit=symbolic_circuit,
    problem_hamiltonian=hamiltonian,
    n_shots_per_circuit=500,
    n_iterations=50,
    target=qnx.QuantinuumConfig(device_name="H1-1LE"),
    use_mbqc=False,  # Disable MBQC
)

result_gates = minimize(
    objective_gates,
    initial_parameters,
    method="COBYLA",
    options={"disp": True, "maxiter": 4},
    tol=1e-2,
)

print()
print("=" * 60)
print("Comparison:")
print(f"  MBQC energy:       {result.fun:.6f}")
print(f"  Gate-based energy: {result_gates.fun:.6f}")
print(f"  Difference:        {abs(result.fun - result_gates.fun):.6f}")
print("=" * 60)

Running gate-based VQE (no MBQC conversion)...


/var/folders/nl/ksgjccyn2jz4qr__r590pykh0000gp/T/ipykernel_37520/1859478437.py:25: DeprecationWarning: The `BackendResult.get_distribution()` method is deprecated: please use `get_empirical_distribution()` or `get_probability_distribution()` instead.
  distribution = results[index].get_distribution()


    Energy: -0.236801
    Energy: 0.277696
    Energy: 0.118618
    Energy: 0.262582

Comparison:
  MBQC energy:       -0.234749
  Gate-based energy: -0.236801
  Difference:        0.002052

   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    4   F =-2.368012E-01    MAXCV = 0.000000E+00
   X = 7.442912E-01   5.364168E-01   3.590919E-01   9.555951E-01
